In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
from collections import Counter
import os
from datetime import datetime
from functools import reduce
import plotly.express as px

In [3]:
METRICS_DATA_DIR = Path.home()/'eqa-dash-data'

In [4]:
def isTrueMetrics(metrics_file_path):
    MANDATORY_SHEETS = ['Hours', 'Scripts', 'Comments']
    df = pd.read_excel(metrics_file_path, None)

    if sorted(MANDATORY_SHEETS) != sorted(df.keys()):
        return False
    
    df_sheet = pd.read_excel(metrics_file_path, sheet_name='Hours')
    for col in ['Date', 'Day', 'Hours']:
        if not col in df_sheet: return False 
    
    df_sheet = pd.read_excel(metrics_file_path, sheet_name='Scripts')
    for col in ['Date', 'Day']:
        if not col in df_sheet: return False 
    
    return True

In [5]:
files_xlsx = [f for f in METRICS_DATA_DIR.iterdir()
              if ((f.is_file()) & (f.suffix == '.xlsx') & (f.stem[:2] != '~$'))
            ]
files_xlsx = list(filter(lambda f: isTrueMetrics(f), files_xlsx))
df_persons = {}

In [6]:
files_xlsx

[PosixPath('/Users/sridhariyer/eqa-dash-data/Felipe_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Priya_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Rajesh_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Venkatesh_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Praveen_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Sasi_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Karthik_Metrics.xlsx'),
 PosixPath('/Users/sridhariyer/eqa-dash-data/Abhishek_Metrics.xlsx')]

In [7]:
len(files_xlsx)

8

In [ ]:
start_date = '01-01-2020'
end_date = '12-31-2020'

In [ ]:
def isTrueMetrics(metrics_file_path):
    MANDATORY_SHEETS = ['Hours', 'Scripts', 'Comments']
    df = pd.read_excel(metrics_file_path, None)

    if sorted(MANDATORY_SHEETS) != sorted(df.keys()):
        print(f'{sorted(MANDATORY_SHEETS)} -- {sorted(df.keys())}')
        return False
    
    df_sheet = pd.read_excel(metrics_file_path, sheet_name='Hours')
    for col in ['Date', 'Day', 'Hours']:
        if not col in df_sheet: return False 
    
    df_sheet = pd.read_excel(metrics_file_path, sheet_name='Scripts')
    for col in ['Date', 'Day']:
        if not col in df_sheet: return False 
    
    return True

In [ ]:
def get_df_total_hours(start_date, end_date):
    for f in files_xlsx:
        if isTrueMetrics(f):
            person_name = f.stem[:f.stem.index('_')]
            df_person = pd.read_excel(f, index_col='Date', sheet_name='Hours')
            df_person = df_person.loc[start_date:end_date]
            df_person.drop(['Day'], axis=1, inplace=True)
            df_person.drop(['Hours'], axis=1, inplace=True)
            float_cols = df_person.select_dtypes(include=['float64']).columns
            str_cols = df_person.select_dtypes(include=['object']).columns
            df_person.loc[:,
                          float_cols] = df_person.loc[:, float_cols].fillna(0)
            df_person.loc[:,
                          str_cols] = df_person.loc[:, str_cols].fillna('')

            df_persons[person_name] = df_person

    # Combinaing all XLSX df into 1 and summing the project hours
    df_hours_series = reduce(lambda x, y: x.add(y, fill_value=0),
                             [df for df in df_persons.values()])
    
    df_total = pd.DataFrame(
        {
            'Projects': pd.Series(df_hours_series.sum().index),
            'Hours': pd.Series(df_hours_series.sum().values)
        }
    )
    
    df_total = df_total[df_total['Hours'] > 0]
    
    return df_total

In [ ]:
df_total_hours = get_df_total_hours(start_date, end_date)
df_total_hours

In [ ]:
len(df_total_hours)

In [ ]:
fig = px.bar(df_total_hours, x='Projects',y='Hours')
fig.update_layout(
#     title="Total Hours Per Project in Q2-2020",
    title=f"Total Hours Per Project from {start_date} to {end_date}",
    autosize=False,
    width=1000,
    height=800,
    xaxis = dict(
    title_text=""))
fig.update_xaxes(automargin=True)
fig.show()

In [ ]:
# --------------------------------------------------------------------------------
# Adding 'Type' to the df_total_hours
# --------------------------------------------------------------------------------
df_total_hours['Type'] = df_total_hours['Projects'].str.replace(
    '-', ' ').str.split(n=1, expand=True)[0]
df_total_hours['Projects'] = df_total_hours['Projects'].str.split('-', n=1).str[1].str.strip()
df_total_hours

In [ ]:
# del df_projtype

In [ ]:
# --------------------------------------------------------------------------------
# Get hours and count for each type of project
# --------------------------------------------------------------------------------

df_projtype = (df_total_hours.groupby('Type')['Hours']
                        .agg(Hours='sum', Count='count')
                        .reset_index()
              ).set_index('Type')
df_projtype

In [ ]:
# --------------------------------------------------------------------------------
# Project type hours pie
# --------------------------------------------------------------------------------
fig2 = px.pie(df_projtype,
              values='Hours',
              names=df_projtype.index,
              title=f'Hours spent by Project Type from {start_date} to {end_date}')

fig2.update_traces(textposition='inside',
                   textinfo='value+label',
                   automargin=True)

In [ ]:
import plotly.graph_objects as go

core_df = df_total_hours[(df_total_hours['Type'] == 'Core')]['Projects']
mobile_df = df_total_hours[(df_total_hours['Type'] == 'Mobile')]['Projects']
poc_df = df_total_hours[(df_total_hours['Type'] == 'POC')]['Projects']

fig = go.Figure(data=[go.Table(
    header=dict(values=list(['Core Projects', 'Mobile Projects', 'POCs']),
                fill_color='paleturquoise',
                align='left',
                font_size=18,
                height=38
               ),
    cells=dict(values=[core_df,mobile_df,poc_df],
               fill_color='lavender',
               align='left',
                font_size=16,
                height=35
              )
    
)
])

fig.update_layout(
    autosize=True,
    height=(max([len(core_df), len(mobile_df), len(poc_df)])/2)*100,
)

fig.show()

In [ ]:
(max([len(core_df), len(mobile_df), len(poc_df)])/2)*100

In [ ]:
fig3 = px.bar(df_projtype, y="Count", text=df_projtype.index)
fig3.update_layout(
    title=f"Number of Project Types from {start_date} to {end_date}",
    autosize=True,
    xaxis={
        'showticklabels':False
    },
    margin={
#         'l':50,
#         'r':50,
#         'b':100,
        't':100,
#         'pad':4
        }
)

fig3.update_traces(textposition='inside')
fig3.update_xaxes(automargin=True)

In [ ]:
[range(1,5)]